![alt text](track.jpg "Title")

<div>
<img src="small_track.png" width="400"/>
</div>


In [ ]:
%%clingo 0 environment.lp -


#script(python)
from clingo.symbol import String, Number, Tuple_, Function

def cell_options(T, move_left, move_forward, move_right):
    t = T.number
    t_bit = '{0:016b}'.format(t)
    
    options = []
    
    #######################################################
    directions = {
        0: 'n',
        1: 'e',
        2: 's',
        3: 'w',
        4: 'n',
    }
    
    cell = {}
    for i in range(0, 3):
        in_dir = directions[i]
        section = list(t_bit[4*i:4*(i+1)])
        if section.count('1') == 0:
            continue
        if section.count('1') == 1:
            options.append(
                Tuple_([String(in_dir),
                        move_forward,
                        String(directions[section.index('1')])]))
        else:
            for j, move in [(i-1, move_left), (i, move_forward), (i+1, move_right)]:
                if section[j] == 1:
                    options.append(
                        Tuple_([String(in_dir),
                                move,
                                String(direction[j])]))

    #######################################################
    # for i in range(0, 4):
    #     section = t_bit[4*i:4*(i+1)]
    #     if section != '0000':
    #         move = [String(directions[i])]
    #         for j in range(0, 4):
    #             move.append(Number(int(section[j])))
    #         options.append(Tuple_(move))
    #######################################################

    return options
#end.

compass(n, "n").
compass(e, "e").
compass(s, "s").
compass(w, "w").


% at time 0 trains should be found at thier starting position
at(train(ID),(X,Y),Dep,D) :- time(Dep), compass(Dir, D), start(ID,(X,Y),Dep,Dir).

% add actions based on train position and cell options
{ action(train(ID),move_forward,T+1) } :- at(train(ID),(X,Y),T,D), option((X,Y),(_,move_forward,D)).

% where a train has not taken a move, add a wait action
action(train(ID),wait,T) :- option((X,Y),_,_,D), time(T), train(ID), not action(train(ID),move_forward,T).

% only one action per train and time T
:- M1 != M2, action(train(ID),M1,T), action(train(ID),M2,T).

% only one at per train and time T
:- D1 != D2, at(train(ID),(_,_),T,D1), at(train(ID),(_,_),T,D2).






option((X,Y), @cell_options(Track, move_left, move_forward, move_right)) :- cell((X,Y), Track).
% option((X,Y), @cell_options(Track, move_left, move_forward, move_right)) :- cell((X,Y), Track).
% option(@cell_options(Track1, Track2, X, Y1, X, Y2)) :- Y1 = Y2 - 1 , cell((X,Y1), Track1), cell((X,Y2), Track2).
% option(@cell_options(Track1, Track2, X1, Y, X2, Y)) :- X1 = X2 - 1 , cell((X1,Y), Track1), cell((X2,Y), Track2).

%#show test/1.
%#show option/2.




clingo version 5.6.2
Reading from environment.lp ...
Solving...
Answer: 1

Answer: 2
action(train(1),move_forward,1)
SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.091s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.045s


-:66:29-48: info: atom does not occur in any rule head:
  option((#X0,#X1),#P2,#P3,#X4)



In [ ]:

% at time 0 trains should be found at thier starting position
at(train(ID),(X,Y),Dep,Dir) :- time(Dep), start(ID,(X,Y),Dep,Dir).

% add actions based on train position and cell options
{ action(train(ID),move_forward,T+1) } :- at(train(ID),(X,Y),T,D), option((X,Y),_,move_forward,D).

% where a train has not taken a move, add a wait action
action(train(ID),wait,T) :- option((X,Y),_,_,D), time(T), train(ID), not action(train(ID),move_forward,T).

% only one action per train and time T
:- M1 != M2, action(train(ID),M1,T), action(train(ID),M2,T).

% only one at per train and time T
:- D1 != D2, at(train(ID),(_,_),T,D1), at(train(ID),(_,_),T,D2).


% if moving forward add ats to indicate new cell positions
at(train(ID),(X2,Y),T2,D) :- X2 = X1-1, option((X1,Y),n,move_forward,D), cell((X2,Y),_),
    action(train(ID),move_forward,T2), at(train(ID),(X1,Y),T,_), T2 = T1 + 1, time(T2), time(T1). 
at(train(ID),(X,Y2),T2,D) :- Y2 = Y1+1, option((X,Y1),e,move_forward,D), cell((X,Y2),_),
    action(train(ID),move_forward,T2), at(train(ID),(X,Y1),T,_), T2 = T1 + 1, time(T2), time(T1).
at(train(ID),(X2,Y),T2,D) :- X2 = X1+1, option((X1,Y),s,move_forward,D), cell((X2,Y),_),
    action(train(ID),move_forward,T2), at(train(ID),(X1,Y),T,_), T2 = T1 + 1, time(T2), time(T1).
at(train(ID),(X,Y1),T2,D) :- Y2 = Y1-1, option((X,Y1),w,move_forward,D), cell((X,Y2),_),
    action(train(ID),move_forward,T2), at(train(ID),(X,Y1),T,_), T2 = T1 + 1, time(T2), time(T1). 

% or if waiting add ats at same position for new time T
at(train(ID),(X,Y),T+1,D) :- time(T+1), time(T), at(train(ID),(X,Y),T,D), action(train(ID),wait,T).


% only allow movement between start and end time
% :- 1 { T > Arr ; T < Dep } 1 action(train(ID), _, T), start(ID,(_,_),Dep,_), end(ID,(X,Y),Arr).

% cell options: based on Track what movements are possible
option((X,Y),n,move_forward,w) :- cell((X,Y),72).
option((X,Y),e,move_forward,s) :- cell((X,Y),72).

option((X,Y),n,move_forward,e) :- cell((X,Y),2064).
option((X,Y),w,move_forward,s) :- cell((X,Y),2064).

option((X,Y),n,move_forward,n) :- cell((X,Y),32800).
option((X,Y),s,move_forward,s) :- cell((X,Y),32800).

% option((X,Y),D,wait,D) :- option((X,Y),_,_,D), cell((X,Y),_).



#show at/4.
#show action/3.
#show option/4.


% TODO:
% + at each time T trains can either move or stay still
% - trains can't be on the same cell
% - trains need to get to end position


% option((X,Y),n,move_forward,n) :- cell((X,Y),33825).
% option((X,Y),e,move_forward,e) :- cell((X,Y),33825).
% option((X,Y),s,move_forward,s) :- cell((X,Y),33825).
% option((X,Y),w,move_forward,w) :- cell((X,Y),33825).
, 

: 

# Notes

cell: `cell((X,Y), Track)`\
train: `train(ID)`\
    start: `start(ID,(X,Y),Dep,Dir)`\
    end: `end(ID,(X,Y),Arr)`\
action:
`action(train(ID), move_left, T)`\
`action(train(ID), move_right, T)`\
`action(train(ID), wait, T)`\
`action(train(ID), move_forward, T)`\


- The track types have been added to this section as a new reference file
- In the start(ID,(X,Y),Dep,Dir) fact, the Dir variable is the direction it faces {n,e,s,w}
- In the start(ID,(X,Y),Dep,Dir) fact, the Dep variable is the earliest departure time
- In the end(ID,(X,Y),Arr) fact, the Arr variable is the latest arrival time
- Therefore, Dep - Arr forms a time window during which the train is permitted to move
- The ending direction of the train is not important—what matters is that it reaches its destination


From the presentation:
- don't need to include the option to wait at every cell, more efficient to skip over them
- never in a situation where a train would have the option to go left, forward, and right. if a train can go forward and left, the forward could also be considered going right, this distinction is up to us
- no deadends, no turning around, also no sharing a cell even in the case of two curved tracks sharing a cell but not touching each other (which were also removed)
- in flatlands the little pink/red building (train station?) indicate where the trains will end, from looking at a png of a geenerated track there is no clear way to know where the trains start
- ALSO axes are switched: top horizontal axis is Y, left vertical is X, top left is (0,0), bottom right is (n,n)
- first action actually spawns the train onto the map, need to add an action for the train to be added.  So time T = 1 the train will be at the starting position?

